In [1]:
# Load Qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Batch
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
API_INSTANCE = os.getenv("API_INSTANCE", None)
service = QiskitRuntimeService(channel="ibm_cloud", token=API_TOKEN, instance=API_INSTANCE)
backend = service.backend(name="ibm_rensselaer")

In [2]:
"""
Example usage of the synthesis benchmarking framework.

This script demonstrates how to:
1. Set up different synthesis compilers
2. Run benchmarks on problem instances
3. Compare results across compilers
4. Store and retrieve results from the database
"""

import logging
from pathlib import Path

# Import the benchmarking library
import benchmarklib
from benchmarklib import BenchmarkDatabase
from benchmarklib.problems import CliqueProblem 
from benchmarklib.compilers import (
    SynthesisBenchmark,
    compare_compilers,
    ClassiqCompiler,
    SynthesisTrial
)

def setup_logging():
    """Configure logging for the benchmark."""
    benchmarklib.setup_logging(
        level=logging.INFO,
        module_levels={
            "synthesis": logging.DEBUG,
            "synthesis.compilers": logging.INFO,
        }
    )



def main():
    """Run synthesis benchmarking example."""
    
    # Setup
    setup_logging()
    print("=" * 60)
    print("SYNTHESIS BENCHMARKING EXAMPLE")
    print("=" * 60)
    
    # Create database for clique problems
    db_path = "Clique.db"
    db_manager = BenchmarkDatabase(
        db_name=db_path,
        problem_class=CliqueProblem,
        trial_class=SynthesisTrial,
    )
    
    print(f"\nDatabase: {db_path}")
    print(f"Problem type: {db_manager.problem_type}")
    
    # ========== STEP 1: Set up compilers ==========
    print("\n" + "="*60)
    print("STEP 1: Setting up synthesis compilers")
    print("="*60)
    
    compilers = [
        # Truth table synthesis (warning: exponential scaling!)
        ClassiqCompiler(),
    ]
    
    print(f"Initialized {len(compilers)} compilers:")
    for compiler in compilers:
        print(f"  - {compiler.name}")
    
    # ========== STEP 2: Create/load problem instances ==========
    print("\n" + "="*60)
    print("STEP 2: Creating problem instances")
    print("="*60)
    
    problems = []
    for n in range(3,13):
        size_filter = {"num_vertices": n}
        problems_subset = db_manager.find_problem_instances(
            size_filters = size_filter,
            choose_untested=True, 
            random_sample=True
        )
        filtered_problems = list(filter(lambda p: p.edge_probability > 50, problems_subset))
        problems += filtered_problems
    
    print(f"\nCreated/loaded {len(problems)} problem instances:")
    for p in problems[:5]:  # Show first 5
        size_info = p.get_problem_size()
        print(f"  - Instance {p.instance_id}: {size_info}")
    
    # ========== STEP 3: Run benchmarks ==========
    print("\n" + "="*60)
    print("STEP 3: Running synthesis benchmarks")
    print("="*60)
    
    # Create benchmark runner
    benchmark = SynthesisBenchmark(
        db_manager=db_manager,
        compilers=compilers,
        save_to_db=True,          # Save results to database
        backend=backend            # backend to transpile to for accurate gate counts
    )
    
    # Run on a subset of problems with specific clique size
    test_problems = problems[:3]  # Use first 3 problems for quick test
    clique_size = 3  # Look for 3-cliques
    
    print(f"\nBenchmarking {len(compilers)} compilers on {len(test_problems)} problems")
    print(f"Target clique size: {clique_size}")
    
    results = benchmark.run_benchmarks(
        problems=test_problems,
        clique_size=clique_size,
        skip_existing=True  # Don't re-run if already in database
    )
    
    # ========== STEP 4: Display results ==========
    print("\n" + "="*60)
    print("STEP 4: Results summary")
    print("="*60)
    
    benchmark.print_summary(results)
    
    # ========== STEP 5: Compare compilers ==========
    print("\n" + "="*60)
    print("STEP 5: Compiler comparison from database")
    print("="*60)
    
    # Compare specific compilers using stored results
    compiler_names_to_compare = [
        "XAG_OPT",
        "TRUTH_TABLE",
    ]
    
    compare_compilers(
        db_manager=db_manager,
        compiler_names=compiler_names_to_compare,
        problem_filters={"num_vertices": 4},  # Only 4-vertex graphs
        clique_size=clique_size
    )
    
    # ========== STEP 6: Database statistics ==========
    print("\n" + "="*60)
    print("STEP 6: Database statistics")
    print("="*60)
    
    stats = db_manager.get_statistics()
    print(f"\nDatabase contains:")
    print(f"  - {stats['problem_instances']} problem instances")
    print(f"  - {stats['trials']['total']} total trials")
    print(f"  - {stats['trials']['completed']} completed trials")
    
    # ========== STEP 7: Advanced analysis ==========
    print("\n" + "="*60)
    print("STEP 7: Advanced analysis example")
    print("="*60)
    
    # Find the most efficient compiler for small problems
    print("\nFinding most efficient compiler for 4-vertex graphs...")
    
    best_by_metric = {
        'synthesis_time': (None, float('inf')),
        'num_qubits': (None, float('inf')),
        'circuit_depth': (None, float('inf')),
        'cx_count': (None, float('inf'))
    }
    
    for compiler_name in compiler_names_to_compare:
        trials = db_manager.find_trials(
            job_id=f"SYNTHESIS_{compiler_name}",
            trial_params={'clique_size': clique_size}
        )
        
        if trials:
            # Average metrics across trials
            metrics = {
                'synthesis_time': [],
                'num_qubits': [],
                'circuit_depth': [],
                'cx_count': []
            }
            
            for trial in trials:
                if trial.counts and isinstance(trial.counts, dict):
                    for metric in metrics:
                        if metric in trial.counts and trial.counts[metric] is not None:
                            metrics[metric].append(trial.counts[metric])
            
            # Update best compilers
            for metric, values in metrics.items():
                if values:
                    avg_value = sum(values) / len(values)
                    if avg_value < best_by_metric[metric][1]:
                        best_by_metric[metric] = (compiler_name, avg_value)
    
    print("\nBest compilers by metric:")
    for metric, (compiler, value) in best_by_metric.items():
        if compiler:
            print(f"  - {metric}: {compiler} ({value:.3f})")
    
    print("\n" + "="*60)
    print("Synthesis benchmarking complete!")
    print("="*60)


In [3]:
#main()

In [4]:
def looped_main():
    """Run synthesis benchmarking example."""
    import math
    
    # Setup
    setup_logging()
    print("=" * 60)
    print("SYNTHESIS BENCHMARKING EXAMPLE")
    print("=" * 60)
    
    # Create database for clique problems
    db_path = "Benchmark.db"
    db_manager = BenchmarkDatabase(
        db_name=db_path,
        problem_class=CliqueProblem,
        trial_class=SynthesisTrial,
    )
    
    print(f"\nDatabase: {db_path}")
    print(f"Problem type: {db_manager.problem_type}")
    
    # Set up compilers
    compilers = [
        ClassiqCompiler()
    ]
    
    print(f"Initialized {len(compilers)} compilers:")
    for compiler in compilers:
        print(f"  - {compiler.name}")
    
    # Create benchmark runner
    benchmark = SynthesisBenchmark(
        db_manager=db_manager,
        compilers=compilers,
        save_to_db=True,
        backend=backend
    )
    
    # ========== MAIN BENCHMARKING LOOP ==========
    all_results = {}
    
    for n in range(3, 20):  # 3 to 12 vertices
        print("\n" + "="*60)
        print(f"BENCHMARKING N={n} VERTICES")
        print("="*60)
        
        # Find problems for this vertex count
        problems = db_manager.find_problem_instances(
            size_filters={"num_vertices": n},
            random_sample=True,
        )
        
        # Filter for edge probability > 50
        problems = [p for p in problems if p.edge_probability > 50]
        
        if not problems:
            print(f"No suitable problems found for n={n}, skipping...")
            continue
        
        # Set clique size to ceiling(n/2)
        clique_size = math.ceil(n / 2)
        
        print(f"Found {len(problems)} problems with edge_probability > 50")
        print(f"Target clique size: {clique_size}")
        
        # Run benchmarks for this n
        results = benchmark.run_benchmarks(
            problems=problems,
            clique_size=clique_size,
            skip_existing=True
        )
        
        all_results[n] = results
        
        # Display results for this n
        print(f"\nResults for n={n}:")
        benchmark.print_summary(results)
    
    # ========== OVERALL SUMMARY ==========
    print("\n" + "="*60)
    print("OVERALL SUMMARY BY VERTEX COUNT")
    print("="*60)
    
    for compiler in compilers:
        print(f"\n{compiler.name} scaling:")
        print("n\tClique\tSuccess\tQubits\tDepth\tCX Count\tTime(s)")
        print("-"*60)
        
        for n in range(3, 13):
            if n not in all_results:
                continue
                
            clique_size = math.ceil(n / 2)
            compiler_results = all_results[n].get(compiler.name, [])
            
            if compiler_results:
                successful = [r for r in compiler_results if r.success]
                if successful:
                    avg_qubits = sum(r.num_qubits for r in successful) / len(successful)
                    avg_depth = sum(r.circuit_depth for r in successful) / len(successful)
                    avg_cx = sum(r.cx_count for r in successful) / len(successful)
                    avg_time = sum(r.synthesis_time for r in successful) / len(successful)
                    success_rate = len(successful) / len(compiler_results) * 100
                    
                    print(f"{n}\t{clique_size}\t{success_rate:.0f}%\t"
                          f"{avg_qubits:.1f}\t{avg_depth:.0f}\t"
                          f"{avg_cx:.0f}\t{avg_time:.3f}")
                else:
                    print(f"{n}\t{clique_size}\tFAILED")
    
    # Database statistics
    stats = db_manager.get_statistics()
    print(f"\nFinal database statistics:")
    print(f"  - {stats['problem_instances']} problem instances")
    print(f"  - {stats['trials']['total']} total trials")

In [ ]:
looped_main()

benchmarklib.core.database - INFO - Database initialized: Benchmark.db (CLIQUE)
benchmarklib.compiler - INFO - SynthesisBenchmark initialized with 1 compilers
benchmarklib.compiler - INFO - Processing problem 33: CLIQUE(num_vertices=3, num_edges=3, edge_probability=65)
benchmarklib.compiler - INFO -   [1/250] Running CLASSIQ...


SYNTHESIS BENCHMARKING EXAMPLE

Database: Benchmark.db
Problem type: CLIQUE
Initialized 1 compilers:
  - CLASSIQ

BENCHMARKING N=3 VERTICES
Found 250 problems with edge_probability > 50
Target clique size: 2
1


benchmarklib.compiler - INFO - Processing problem 1323: CLIQUE(num_vertices=3, num_edges=3, edge_probability=93)
benchmarklib.compiler - INFO -   [2/250] Running CLASSIQ...


1


benchmarklib.compiler - INFO - Processing problem 1297: CLIQUE(num_vertices=3, num_edges=3, edge_probability=89)
benchmarklib.compiler - INFO -   [3/250] Running CLASSIQ...


...

(((((((((((((((((((((((((((1 & (~((vertices[0]) & (vertices[4])))) & (~((vertices[0]) & (vertices[6])))) & (~((vertices[0]) & (vertices[7])))) & (~((vertices[0]) & (vertices[9])))) & (~((vertices[1]) & (vertices[4])))) & (~((vertices[1]) & (vertices[5])))) & (~((vertices[1]) & (vertices[6])))) & (~((vertices[1]) & (vertices[7])))) & (~((vertices[1]) & (vertices[11])))) & (~((vertices[1]) & (vertices[12])))) & (~((vertices[2]) & (vertices[9])))) & (~((vertices[2]) & (vertices[11])))) & (~((vertices[2]) & (vertices[13])))) & (~((vertices[3]) & (vertices[13])))) & (~((vertices[4]) & (vertices[10])))) & (~((vertices[4]) & (vertices[13])))) & (~((vertices[5]) & (vertices[10])))) & (~((vertices[5]) & (vertices[13])))) & (~((vertices[6]) & (vertices[11])))) & (~((vertices[6]) & (vertices[12])))) & (~((vertices[7]) & (vertices[8])))) & (~((vertices[7]) & (vertices[10])))) & (~((vertices[8]) & (vertices[10])))) & (~((vertices[8]) & (vertices[12])))) & (~((vertices[9]) & (vertices[11])))) & (~((

benchmarklib.compiler - INFO - Processing problem 586: CLIQUE(num_vertices=14, num_edges=60, edge_probability=71)
benchmarklib.compiler - INFO -   [17/250] Running CLASSIQ...


((((((((((((((((((((((((((((((1 & (~((vertices[0]) & (vertices[4])))) & (~((vertices[0]) & (vertices[7])))) & (~((vertices[0]) & (vertices[9])))) & (~((vertices[0]) & (vertices[11])))) & (~((vertices[1]) & (vertices[2])))) & (~((vertices[1]) & (vertices[3])))) & (~((vertices[1]) & (vertices[7])))) & (~((vertices[1]) & (vertices[8])))) & (~((vertices[1]) & (vertices[10])))) & (~((vertices[1]) & (vertices[12])))) & (~((vertices[2]) & (vertices[5])))) & (~((vertices[2]) & (vertices[9])))) & (~((vertices[3]) & (vertices[10])))) & (~((vertices[3]) & (vertices[12])))) & (~((vertices[4]) & (vertices[7])))) & (~((vertices[4]) & (vertices[8])))) & (~((vertices[4]) & (vertices[11])))) & (~((vertices[4]) & (vertices[12])))) & (~((vertices[4]) & (vertices[13])))) & (~((vertices[5]) & (vertices[12])))) & (~((vertices[6]) & (vertices[7])))) & (~((vertices[7]) & (vertices[9])))) & (~((vertices[7]) & (vertices[11])))) & (~((vertices[7]) & (vertices[12])))) & (~((vertices[7]) & (vertices[13])))) & (~((

benchmarklib.compiler - INFO - Processing problem 6123: CLIQUE(num_vertices=14, num_edges=60, edge_probability=61)
benchmarklib.compiler - INFO -   [18/250] Running CLASSIQ...


((((((((((((((((((((((((((((((1 & (~((vertices[0]) & (vertices[4])))) & (~((vertices[0]) & (vertices[12])))) & (~((vertices[0]) & (vertices[13])))) & (~((vertices[1]) & (vertices[9])))) & (~((vertices[1]) & (vertices[13])))) & (~((vertices[2]) & (vertices[3])))) & (~((vertices[2]) & (vertices[4])))) & (~((vertices[2]) & (vertices[6])))) & (~((vertices[2]) & (vertices[8])))) & (~((vertices[2]) & (vertices[10])))) & (~((vertices[2]) & (vertices[11])))) & (~((vertices[2]) & (vertices[13])))) & (~((vertices[3]) & (vertices[4])))) & (~((vertices[3]) & (vertices[7])))) & (~((vertices[3]) & (vertices[11])))) & (~((vertices[3]) & (vertices[12])))) & (~((vertices[4]) & (vertices[12])))) & (~((vertices[4]) & (vertices[13])))) & (~((vertices[5]) & (vertices[7])))) & (~((vertices[5]) & (vertices[8])))) & (~((vertices[5]) & (vertices[9])))) & (~((vertices[5]) & (vertices[10])))) & (~((vertices[6]) & (vertices[11])))) & (~((vertices[6]) & (vertices[12])))) & (~((vertices[7]) & (vertices[10])))) & (~

In [ ]:
db_path = "Clique.db"
db_manager = BenchmarkDatabase(
    db_name=db_path,
    problem_class=CliqueProblem,
    trial_class=SynthesisTrial,
)

size_filter = {"num_vertices": 5, 'edge_probability': 53}
problem = db_manager.find_problem_instances(
    size_filters = size_filter,
    random_sample=True,
    limit=1
)[0]

compiler = ClassiqCompiler()

compiler.compile(problem, clique_size=3)

benchmarklib.quantum_trials - INFO - Database initialized: Clique.db (CLIQUE)


AttributeError: 'FrameInfo' object has no attribute 'positions'

In [ ]:
from classiq import *
from math import ceil, log2
import networkx as nx

@qfunc
def main(vertices: Output[QArray[QNum[1]]], oracle_result: Output[QNum[1]]):
    """
    Main function for Grover's algorithm setup.
    """
    # Allocate and initialize vertices
    allocate(G.number_of_nodes(), vertices)

    # Allocate oracle result qubit
    cond = 1

    for i in range(G.number_of_nodes()):
        for j in range(i + 1, G.number_of_nodes()):
            if not G.has_edge(i, j):
                cond &= ~(vertices[i] & vertices[j])

    # Properly allocate and initialize counter
    COUNTER_SIZE = ceil(log2(G.number_of_nodes() + 1))  # Need to count from 0 to N
    counter = QNum("counter", COUNTER_SIZE)  # Allocate with proper size
    allocate(COUNTER_SIZE, counter)

    for i in range(vertices.len):
        counter += vertices[i]

    cond_count = counter >= clique_size
    cond &= cond_count

    oracle_result |= cond

    for i in range(vertices.len):
        counter += -vertices[i]

G = nx.from_numpy_array(problem.as_adjacency_matrix())
clique_size = 3

qmod = create_model(main)
qprog = synthesize(qmod)
